# Quick Start
In this notebook, we go over the main functionalities of the library

## Table of Contents
1. [Installation](#installation)
2. [Importing Libraries](#importing-libraries)
3. [Creating Sample Data](#creating-sample-data)
4. [Initializing EvaluationManager](#initializing-evaluationmanager)
5. [Evaluating Predictions](#evaluating-predictions)

## Installation

We recommend using some virtual environment

With pip

`pip install views-evaluation`

## Importing Libraries
First, let's import a few things:

In [1]:
import pandas as pd
import numpy as np
from views_evaluation.evaluation.evaluation_manager import EvaluationManager

## Creating Sample Data

Let's create some sample data for actual values and predictions.

In [2]:
index = pd.MultiIndex.from_tuples(
    [(99, 1), (99, 2), (100, 1), (100, 2), (101, 1), (101, 2), (102, 1), (102, 2)],
    names=["month", "country"],
)
index_0 = pd.MultiIndex.from_tuples(
    [(100, 1), (100, 2), (101, 1), (101, 2)],
    names=["month", "country"],
)
index_1 = pd.MultiIndex.from_tuples(
    [(101, 1), (101, 2), (102, 1), (102, 2)],
    names=["month", "country"],
)

In [7]:
# Actual data
df_actual = pd.DataFrame(
    {
        "lr_target": [0, 1, 1, 2, 2, 3, 3, 4],
        "covariate_1": [3, 2, 4, 5, 2, 6, 8, 5],
    },
    index=index,
)

In [8]:
# Point predictions
df1_point = pd.DataFrame({"pred_lr_target": [1, 3, 5, 7]}, index=index_0)
df2_point = pd.DataFrame({"pred_lr_target": [2, 4, 6, 8]}, index=index_1)
dfs_point = [df1_point, df2_point]

# Uncertainty
df1_uncertainty = pd.DataFrame(
    {"pred_lr_target": [[1, 2, 3], [2, 3, 4], [3, 4, 5], [4, 5, 6]]}, index=index_0
)
df2_uncertainty = pd.DataFrame(
    {"pred_lr_target": [[4, 6, 8], [5, 7, 9], [6, 8, 10], [7, 9, 11]]}, index=index_1
)
dfs_uncertainty = [df1_uncertainty, df2_uncertainty]

## Initializing EvaluationManager
Now, we can initialize the `EvaluationManager` with the metrics we want to evaluate.
Point evaluation supports the following metrics:
- RMSLE
- CRPS
- Average Precision

Uncertainty evaluation supports the following metric:
- CRPS

In [18]:
metrics_list = ['RMSLE', 'CRPS', 'MIS'] # Add other metrics as needed
evaluation_manager = EvaluationManager(metrics_list)

## Evaluating Predictions

In [19]:
steps = [1, 2]
point_evaluation_results = evaluation_manager.evaluate(df_actual, dfs_point, target='lr_target', steps=steps)

Metric MIS is not a default metric, skipping...
Metric MIS is not a default metric, skipping...
Metric MIS is not a default metric, skipping...


In [20]:
point_evaluation_results['month'][1], point_evaluation_results['step'][1], point_evaluation_results['time_series'][1]

(             RMSLE  CRPS
 month100  0.203422   0.5
 month101  0.502668   2.0
 month102  0.573874   3.5,
            RMSLE  CRPS
 step01  0.182040   0.5
 step02  0.636311   3.5,
          RMSLE  CRPS
 ts00  0.510800   2.0
 ts01  0.420849   2.0)

Metrics will be **ignored** if not in the supported metric list

In [21]:
uncertainty_evaluation_results = evaluation_manager.evaluate(df_actual, dfs_uncertainty, target='lr_target', steps=steps)

Metric RMSLE is not a default metric, skipping...


Metric RMSLE is not a default metric, skipping...
Metric RMSLE is not a default metric, skipping...


In [24]:
uncertainty_evaluation_results['month'][1], uncertainty_evaluation_results['step'][1], uncertainty_evaluation_results['time_series'][1]

(              CRPS     MIS
 month100  0.555556    3.90
 month101  2.333333   65.85
 month102  4.111111  127.80,
             CRPS    MIS
 step01  1.833333  45.85
 step02  2.833333  85.85,
           CRPS    MIS
 ts00  1.055556   23.9
 ts01  3.611111  107.8)

If you are only interested in one of the evaluation schemas, you can call the corresponding function

In [16]:
# Get the evaluation type, i.e., uncertainty or point
actual = EvaluationManager.transform_data(
            EvaluationManager.convert_to_arrays(df_actual), 'lr_target'
        )
predictions = [
    EvaluationManager.transform_data(
        EvaluationManager.convert_to_arrays(pred), f"pred_lr_target"
    )
    for pred in dfs_point
]
is_uncertainty = EvaluationManager.get_evaluation_type(predictions)
month_point_evaluation_results = evaluation_manager.month_wise_evaluation(actual, predictions, target='lr_target', is_uncertainty=is_uncertainty)

In [17]:
print(month_point_evaluation_results[1])

             RMSLE  CRPS
month100  0.203422   0.5
month101  0.502668   2.0
month102  0.573874   3.5
